# Scraping de URLs Legítimas – Sector SaaS / Cloud / Correo

**Objetivo:**  
## 1. Objetivo

Documentar el proceso de recopilación de URLs legítimas de los principales servicios SaaS, cloud y correo electrónico, para construir el dataset de entrenamiento del modelo de detección de phishing.

> Nota: En esta fase inicial, se replicó la metodología básica usada en banca, sin aplicar adaptaciones específicas para el sector SaaS/Cloud.

 En esta fase se documentan:
- Las empresas objetivo,
- las URLs relevantes recopiladas (login, acceso, recuperación, etc.),
- los problemas encontrados,
- y los próximos pasos identificados.

La metodología empleada es scraping básico sobre la home/login, sin crawling profundo ni Selenium.  
Las mejoras y adaptaciones específicas se valorarán tras el análisis de resultados de esta primera extracción.

## 1. Lista de empresas/servicios objetivo

| Empresa/Servicio | URL Principal                       | Sector/Tipo    | Observaciones             |
|------------------|-------------------------------------|----------------|---------------------------|
| Microsoft        | https://login.microsoftonline.com    | Cloud/Correo   | Top mundial phishing      |
| Google           | https://accounts.google.com          | Cloud/Correo   | Top mundial phishing      |
| Apple            | https://appleid.apple.com            | Cloud/Correo   | Top mundial phishing      |
| Dropbox          | https://www.dropbox.com/login        | Cloud          |                           |
| Adobe            | https://account.adobe.com            | SaaS           |                           |
| Zoom             | https://zoom.us/signin               | SaaS           |                           |
| Slack            | https://slack.com/signin             | SaaS           |                           |
| DocuSign         | https://account.docusign.com         | SaaS           |                           |
| TeamViewer       | https://login.teamviewer.com         | SaaS           |                           |
| Atlassian        | https://id.atlassian.com             | SaaS           |                           |
| Box              | https://account.box.com/login        | Cloud          |                           |
| Cisco Webex      | https://signin.webex.com             | SaaS           |                           |
| Zoho             | https://accounts.zoho.com            | SaaS           |                           |
| GoToMeeting      | https://signin.logmeininc.com        | SaaS           |                           |
| ProtonMail       | https://mail.proton.me/login         | Correo         |                           |
| Fastmail         | https://www.fastmail.com/login       | Correo         |                           |
| Mega             | https://mega.nz/login                | Cloud          |                           |
| Yandex Mail      | https://passport.yandex.com          | Correo         |                           |
| iCloud           | https://www.icloud.com/login         | Cloud/Correo   |                           |
| OVHcloud         | https://www.ovh.com/auth/            | Cloud          |                           |


## 2. Metodología

- **Fase 1:** Scraping básico de la página principal (home/login) de cada servicio, extrayendo únicamente los enlaces internos visibles sin interacción avanzada.
- **Fase 2:** Registro y documentación de problemas encontrados durante el scraping (por ejemplo, páginas minimalistas, enlaces limitados, detección de bots).
- **Fase 3:** Análisis de resultados obtenidos y planteamiento de mejoras para futuras fases (evaluar la necesidad de crawling profundo o el uso de Selenium en servicios con baja cobertura de URLs).


## 3. Resultados iniciales

- **Total de URLs obtenidas:** 72
- **Empresas cubiertas:** 9
- **Problemas detectados:**  
  - Algunas webs (ej. Google, Zoom, Mega, iCloud) presentan una home muy minimalista, con pocos enlaces útiles accesibles.
  - En la mayoría de casos no se detectaron bloqueos activos ni captchas, pero sí una clara limitación en la profundidad de enlaces recogidos.
- **Incidencias:**  
  - El dataset resultante es muy desbalanceado: más del 80% de las URLs provienen de Box y Slack, el resto de empresas aportan muy pocos enlaces útiles.
  - Posible inclusión de enlaces poco relevantes (ayuda, términos de servicio, etc.) debido a la metodología básica empleada.


## 4. Próximos pasos

- Filtrar URLs realmente relevantes para phishing.
- Valorar crawling profundo y uso de Selenium en webs con baja cobertura.
- Ampliar y revisar la lista de empresas en siguientes iteraciones.
- Preparar los datos para limpieza y análisis.


## 5. Registro de problemas y soluciones

- Enlaces útiles limitados en Google, Zoom, Mega e iCloud (pendiente de crawling/Selenium).
- Box y Slack aportan muchas URLs pero con mezcla de rutas poco relevantes (pendiente de filtrado).
- Sin bloqueos ni captchas detectados en esta fase.


## 6. Código y logs relevantes

- Notebook principal: `notebooks/scraping_saas.ipynb`
- Scripts de scraping y crawling: (añadir enlaces si los separas en otros archivos)
- Logs de ejecución: (añadir referencias o archivos si registras errores o incidencias específicas)
- Dataset generado: `data/raw/saas_legitimas_crudo.csv`



In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd, os
import time
from urllib.parse import urljoin, urlparse
from time import sleep

## Lista inicial de empresas

In [2]:
saas_empresas = {
    "Microsoft": "https://login.microsoftonline.com",
    "Google": "https://accounts.google.com",
    "Apple": "https://appleid.apple.com",
    "Dropbox": "https://www.dropbox.com/login",
    "Adobe": "https://account.adobe.com",
    "Zoom": "https://zoom.us/signin",
    "Slack": "https://slack.com/signin",
    "DocuSign": "https://account.docusign.com",
    "TeamViewer": "https://login.teamviewer.com",
    "Atlassian": "https://id.atlassian.com",
    "Box": "https://account.box.com/login",
    "Cisco Webex": "https://signin.webex.com",
    "Zoho": "https://accounts.zoho.com",
    "GoToMeeting": "https://signin.logmeininc.com",
    "ProtonMail": "https://mail.proton.me/login",
    "Fastmail": "https://www.fastmail.com/login",
    "Mega": "https://mega.nz/login",
    "Yandex Mail": "https://passport.yandex.com",
    "iCloud": "https://www.icloud.com/login",
    "OVHcloud": "https://www.ovh.com/auth/"
}


## Función para scrapear

In [3]:
def obtener_urls(base_url, delay=1):
    """
Descarga la página principal y extrae las URLs internas
Acepta URLs del dominio principal y subdominios
"""
    try:
        # Realiza la petición HTTP a la URL de la enpresa con 10s máximos de espera
        response = requests.get(base_url, timeout=10)
        
        # Parsea el HTML recibido
        soup = BeautifulSoup(response.text, 'html.parser')

        # Obtiene el dominio principal
        dominio_empresa = urlparse(base_url).netloc

        # Elimina "www." al inicio para igualar subdominios y dominio principal

        if dominio_empresa.startswith('www.'):
            dominio_base = dominio_empresa[4:]
        else:
            dominio_base = dominio_empresa

        # Set para almacenar URLs únicas
        urls = set()

        # Itera por todos los enlaces (a, href)
        for link in soup.find_all('a', href=True):
            href = link['href'] # Extrae el href del enlace
            href = urljoin(base_url, href) #Convierte relativo a absoluto

            #Obtiene el dominio del enlace
            dominio_href = urlparse(href).netloc
            if dominio_href.startswith('www.'):
                dominio_href_base = dominio_href[4:]
            else:
                dominio_href_base = dominio_href

            # Añade la URL solo si es del dominio principal o subdomino
            if dominio_href_base == dominio_base or dominio_href_base.endswith('.' + dominio_base):
                urls.add(href)

        # Devuelve las URLs únicas como lista
        return list(urls)

    except Exception as e:
        print(f'Error al acceder a {base_url}: {e}')
        return []
 

        

        

    

## Guardamos los resultados 

In [4]:
resultados = []
for nombre, url_base in saas_empresas.items():
    print(f'Scraping empresa: {nombre}')
    urls_empresa = obtener_urls(url_base) 
    for url in urls_empresa:
        resultados.append({'empresa': nombre, 'url': url})
    time.sleep(2)
    

Scraping empresa: Microsoft
Scraping empresa: Google
Scraping empresa: Apple
Scraping empresa: Dropbox
Scraping empresa: Adobe
Scraping empresa: Zoom
Scraping empresa: Slack
Scraping empresa: DocuSign
Scraping empresa: TeamViewer
Scraping empresa: Atlassian
Scraping empresa: Box
Scraping empresa: Cisco Webex
Scraping empresa: Zoho
Scraping empresa: GoToMeeting
Error al acceder a https://signin.logmeininc.com: HTTPSConnectionPool(host='signin.logmeininc.com', port=443): Max retries exceeded with url: / (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x11e44c580>: Failed to resolve 'signin.logmeininc.com' ([Errno 8] nodename nor servname provided, or not known)"))
Scraping empresa: ProtonMail
Scraping empresa: Fastmail
Scraping empresa: Mega
Scraping empresa: Yandex Mail
Scraping empresa: iCloud
Scraping empresa: OVHcloud


In [5]:
# Convertirmos los resultados en un DataFrame
df_scrap = pd.DataFrame(resultados)
#Guardamos el DataFrame en un archivo CSV
df_scrap.to_csv('../data/raw/saas_legitimas_crudo.csv', index=False)

In [6]:
print(df_scrap['empresa'].value_counts())




empresa
Box            30
Slack          28
Google          4
Zoom            3
Fastmail        2
OVHcloud        2
Mega            1
Yandex Mail     1
iCloud          1
Name: count, dtype: int64




**Resumen:**
- Se han recolectado 72 URLs de 9 empresas del sector SaaS/Cloud/Correo.
- El 80% de las URLs pertenecen a Box y Slack, lo que muestra un dataset muy desbalanceado.
- El resto de empresas aportan solo 1-4 URLs cada una, probablemente por falta de enlaces internos o restricciones antirrobots.
- No hay duplicados exactos.
- Las rutas cubren login, ayuda, TOS, accesibilidad, etc. De cara a entrenamiento de modelos de phishing, sería recomendable filtrar por relevancia (`login`, `signin`, `reset`, etc.).

**Limitaciones detectadas:**
- Baja diversidad de empresas.
- Posible inclusión de rutas poco útiles (TOS, ayuda, etc.).
- No se han aplicado técnicas de crawling avanzado ni Selenium.

**Próximos pasos propuestos:**
- Filtrado y priorización de URLs relevantes para phishing.
- Reintentar scraping/crawling más profundo en empresas con baja cobertura.
- Evaluar uso de Selenium para webs que bloquean scraping básico.

**Checkpoint documentado:**
- Fase de scraping/crawling básico cerrada.
- Listo para iterar limpieza y crawling avanzado en segunda vuelta.
